# Lab 10
Understanding Docker Networks

What is Container Networking?
Container networking also called Software Defined Networking (SDN) is what allows Docker networkgin subsystems to work. 

Some defalt Docker Netwrok drivers are:
- Bridge
    - a Network that is User-defined to allow for all containers connected on the same network to communicate with eachother. 
- Host
    - a container is attahed to a Docker host. Any traffic routed to the container must be routed from the host system.
- Overlay
    - Created in a network spanning multiple docker hosts. Think each of your docker containers in the lab talking to eachother
- Macvlan
    - used in complicated networking to assing logical addresses based on MAC addresses to the virtual network interfaces making it apear as if the container has a physical network interaface connected to the network. Used in legacy applications.
- None
    -  the container only has a loopback interface so it can talk to itself.



# Part 1 - Setup

In [4]:
!docker run -d --name nginx --network=none -p 81:81 nginx

docker: Error response from daemon: Conflict. The container name "/nginx" is already in use by container "64fc640ff4ba74f370abb471badb15ebb8b379d34c8faa101417320ae6db0e5b". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


## Note:
**If you get this error:**

*docker: permission denied while trying to connect to the Docker daemon ...*

run this in wsl: 
``` bash
sudo usermod -aG docker $USER
newgrp docker
sudo systemctl restart docker
```
Run it again....

you should get a hash number YAY!

`cbb2f01cd1b...`

--------------------------------------
If you are still getting the error


run this:
``` bash
sudo chmod 666 /usr/docker.sock
```
--------------------------------------
**If you get this error you did things right**

docker: Error response from daemon: Conflict. The container name "/nginx

--------------------------------------


## we now have an nginx container 

NGINX serves static content (like HTML, CSS, JavaScript, images) efficiently, improving website performance.

In [9]:
!curl localhost

#the curl command should fail on our system 

curl: (7) Failed to connect to localhost port 80 after 0 ms: Couldn't connect to server


Lets run it on the docker container 

In [8]:
!docker exec -it nginx2 /bin/sh -c "curl google.com"



Error response from daemon: No such container: nginx2


* curl:(6) Could not resolve host: google.com

As you can see the nginx container can not talk to google.com

# Part 2 - Understanding the network 
Docker comes with a subcommand for handling Docker networks.

In [10]:
!docker network

Usage:  docker network COMMAND

Manage networks

Commands:
  connect     Connect a container to a network
  create      Create a network
  disconnect  Disconnect a container from a network
  inspect     Display detailed information on one or more networks
  ls          List networks
  prune       Remove all unused networks
  rm          Remove one or more networks

Run 'docker network COMMAND --help' for more information on a command.


running `docker network ls` will show us all the names of the networks.


In [12]:
!docker network ls 

NETWORK ID     NAME      DRIVER    SCOPE
3ff9a0de492f   bridge    bridge    local
73b1757b2039   host      host      local
5f6b5826c514   none      null      local


In the above you can see that there is a none network. 

When we setup the docker nginx server we did not give it a network

lets look for the nginx server

In [23]:
!docker network inspect none 

[
    {
        "Name": "none",
        "Id": "f7b0737ec1188b800684be7edfaea4ba5d2871bc12f00933bf55712a110ee27d",
        "Created": "2025-04-18T12:24:26.478234728-07:00",
        "Scope": "local",
        "Driver": "null",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": null
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "cbb2f01cd1b1f17d502470e765159bc3400ca19f6862028bbc0ebb2c065f8183": {
                "Name": "nginx2",
                "EndpointID": "64ba2adbfd4b93f0be8a480f884b4417c5e69f2f221294786f127342b46cf6b8",
                "MacAddress": "",
                "IPv4Address": "",
                "IPv6Address": ""
            }
        },
        "Options": {},
        "Labels": {}
    }
]


## OK Enough of that nonsense
Lets install docker and use the API to do some work this is a python class after all

In [1]:
!pip install docker

In [2]:
import docker

client = docker.from_env()

print("Docker version:", client.version()['Version'])


Docker version: 28.0.4


Lets create a quick code to list the containers using the docker client

In [3]:
def list_running_containers():
    containers = client.containers.list(all=True)# list all containers
    #containers = client.containers.list() #list only running containers
    for container in containers:
        print(f"{container.name} - {container.status}")

list_running_containers()

nginx - running


Here you can see the running container and no surprise it is the nginx we started before


In [4]:
#lest identify the network of the nginx container
def show_container_ip(container_name):
    container = client.containers.get(container_name)
    networks = container.attrs['NetworkSettings']['Networks']
    
    for net_name, net_data in networks.items():
        print(f"{container_name} in network '{net_name}' has IP: {net_data['IPAddress']}")


show_container_ip("nginx")


nginx in network 'none' has IP: 


again no surprise there is no IP address as it does not have a network. 

### ok I lied back to creating more stuff though the CLI

In [5]:
#setup another container for mysql
!docker run -d --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8
#Note the -d means detached mode so no container is using created just the MySQL database 

Unable to find image 'mysql:8' locally
8: Pulling from library/mysql

85fb56d1: Pulling fs layer 
6647a9d0: Pulling fs layer 
50c6393c: Pulling fs layer 
39a1086c: Pulling fs layer 
288abb8b: Pulling fs layer 
c94b602f: Pulling fs layer 
09fc8b3b: Pulling fs layer 
d8b9ec0f: Pulling fs layer 
ce28ec2e: Pulling fs layer 
Digest: sha256:3a89257a3a979e46e3650628bea227a6bc318a2bdc95f954489ec89573a4c6a2[8ADownloading  4.194MB/129.5MBDownloading   34.6MB/46.51MB
Status: Downloaded newer image for mysql:8
58285df5ac08b0076085008482c50109ec0d546867cee7194a53c88805b1dfa2


## The following needs to be run in the terminal

In [6]:
!docker run -d --name adminer -p 8080:8080 adminer


Unable to find image 'adminer:latest' locally
latest: Pulling from library/adminer

48f4eacb: Pulling fs layer 
228eae71: Pulling fs layer 
db1dd7a4: Pulling fs layer 
747c868b: Pulling fs layer 
6b371bd9: Pulling fs layer 
159412d9: Pulling fs layer 
76ac181e: Pulling fs layer 
1e15690e: Pulling fs layer 
b700ef54: Pulling fs layer 
b5b6ce94: Pulling fs layer 
95d66e87: Pulling fs layer 
04a1b939: Pulling fs layer 
2628d500: Pulling fs layer 
8bc8a0cd: Pulling fs layer 
ec9c416f: Pulling fs layer 
Digest: sha256:b1c1395ff805e5ab62dca9ca37dc1ab678436551a706d68e7a77b15b9f7aa266
Status: Downloaded newer image for adminer:latest
e79e3d5ce6d396a0ffd4b654d4da4ee53b8debee06812312017b8fe48649728c


Run the following each in there own terminal

``` bash
docker logs -f mysql
docker logs -f adminer
```

In [7]:
!docker inspect mysql | grep -n1 "IPAddress\":"
# or
print("-"*30)
show_container_ip("mysql")


208-            "GlobalIPv6PrefixLen": 0,
209:            "IPAddress": "172.17.0.2",
210-            "IPPrefixLen": 16,
--
223-                    "Gateway": "172.17.0.1",
224:                    "IPAddress": "172.17.0.2",
225-                    "IPPrefixLen": 16,
------------------------------
mysql in network 'bridge' has IP: 172.17.0.2


In [ ]:
!docker inspect adminer| grep  -n1 "IPAddress\":"
# or
print("-"*30)
show_container_ip("adminer")


220-            "GlobalIPv6PrefixLen": 0,
221:            "IPAddress": "172.17.0.3",
222-            "IPPrefixLen": 16,
--
233-                    "Gateway": "172.17.0.1",
234:                    "IPAddress": "172.17.0.3",
235-                    "IPPrefixLen": 16,
------------------------------
adminer in network 'bridge' has IP: 172.17.0.3


### Lets do a bit of cleanup 

``` bash
#lets remove our old container
!docker stop adminer
!docker rm adminer
!docker stop mysql
!docker rm mysql
!docker stop nginx 
!docker rm nginx
```

In [14]:
def clean_containers():
    containers = client.containers.list(all=True)
    if not containers:
        print("No containers to clean.")
        return
    
    for container in containers:
        container.stop()
        container.reload()
        if container.status == 'exited':
            print(f"Removing {container.name}")
            container.remove()

list_running_containers()

clean_containers()


nginx - exited
Removing nginx


## Setting up the network

In [15]:
# Create a network called yourNetwork using the following command

!docker network create yourNetwork

Error response from daemon: network with name yourNetwork already exists


In [16]:
!docker network inspect yourNetwork

[
    {
        "Name": "yourNetwork",
        "Id": "38bc12c9a366d46934e132005e1b209dcd1db1b45d05b957fb96feb607ba556d",
        "Created": "2025-04-19T04:18:06.007004167Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv4": true,
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": {},
            "Config": [
                {
                    "Subnet": "172.18.0.0/16",
                    "Gateway": "172.18.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {},
        "Options": {},
        "Labels": {}
    }
]


In [17]:
!docker run -d --network yourNetwork --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8


85cfc557cc51d6aa1bb549ba36edea660c350aa3fbea87e185de44fd1721885a


In [18]:
!docker run -d --network yourNetwork --name adminer -p 8080:8080 adminer

2d5da2b994f2db116c0ff5df4fa61ee78b9d92f30cb6a41ee741ed4526041d26


In [19]:
#Now lets connect the adminer
!docker network connect yourNetwork adminer

Error response from daemon: endpoint with name adminer already exists in network yourNetwork


In [20]:
!docker inspect mysql |grep -C2 " \"Networks\""
!docker inspect mysql |grep -A9 " \"Networks\""| grep " \"IPAddress\""
#or
print("-"*30)
show_container_ip("mysql")

            "IPv6Gateway": "",
            "MacAddress": "",
            "Networks": {
                "yourNetwork": {
                    "IPAMConfig": null,
------------------------------
mysql in network 'yourNetwork' has IP: 172.18.0.2


In [21]:
!docker inspect adminer| grep -A2 " \"Networks\""
!docker inspect adminer| grep -A9 " \"Networks\""| grep " \"IPAddress\""
!ping -c 1 172.18.0.3
#or
print("-"*30)
show_container_ip("adminer")

            "Networks": {
                "yourNetwork": {
                    "IPAMConfig": null,
PING 172.18.0.3 (172.18.0.3): 56 data bytes

--- 172.18.0.3 ping statistics ---
1 packets transmitted, 0 packets received, 100.0% packet loss
------------------------------
adminer in network 'yourNetwork' has IP: 172.18.0.3


# LAB 
As a reminder you should be submitting to me 2 files. 
- _YourName_Lab 09.ipynb
    - this is your lab report
- _YourName__\<name of your script\>.py

## Part 1 - warm up

Now that you have 2 containers up and running and know there connected. Create a python script that will check to see if the IP addresses of the containers are up and running. 



In [ ]:
#Create and inspect a network

# Import the Docker library
import docker

# Connect to Docker
client = docker.from_env()

# Create a network
def create_network(network_name):
    networks = client.networks.list(names=[network_name])
    if not networks:
        print(f"Creating network: {network_name}...")
        client.networks.create(network_name, driver="bridge")
    else:
        print(f"Network '{network_name}' already exists.")

# Inspect a network
def inspect_network(network_name):
    network = client.networks.get(network_name)
    print(f"Network Name: {network.name}")
    print(f"Network Driver: {network.attrs['Driver']}")
    print(f"Connected Containers: {len(network.attrs['Containers'])}")

network_name = "yourNetwork"
create_network(network_name)
inspect_network(network_name)




Network 'yourNetwork' already exists.
Network Name: yourNetwork
Network Driver: bridge
Connected Containers: 0



## Part 2
Continue with the above code and create a script that will:
- Relaunch a container if it is ever shutdown. 
- Perform a maintenance restart after 1 day has passed. 
- Monitor the log files and outputs errors and warnings.

your python file does not need to be in a docker container


https://docs.docker.com/engine/api/sdk/examples/



In [ ]:
# Run and connect containers to the network

# Remove a container if it already exists
def remove_container_if_exists(container_name):
    try:
        container = client.containers.get(container_name)
        print(f"Container '{container_name}' already exists. Removing it...")
        container.stop()
        container.remove()
    except docker.errors.NotFound:
        print(f"Container '{container_name}' does not exist. No need to remove.")

# Function to run MySQL container
def run_mysql_container(network_name):
    try:
        print("Running MySQL container...")
        client.containers.run(
            "mysql:8",
            name="mysql",
            environment={"MYSQL_ROOT_PASSWORD": "1"},
            ports={"3306/tcp": 3306},
            network=network_name,
            detach=True
        )
        print("MySQL container started.")
    except docker.errors.APIError as e:
        print("Error with MySQL container:", e.explanation)

# Function to run Adminer container
def run_adminer_container(network_name):
    try:
        print("Running Adminer container...")
        client.containers.run(
            "adminer",
            name="adminer",
            ports={"8080/tcp": 8080},
            network=network_name,
            detach=True
        )
        print("Adminer container started.")
    except docker.errors.APIError as e:
        print("Error with Adminer container:", e.explanation)

# Show container IP address
def show_container_ip(container_name, network_name):
    try:
        container = client.containers.get(container_name)
        ip_address = container.attrs['NetworkSettings']['Networks'][network_name]['IPAddress']
        print(f"{container_name} IP Address: {ip_address}")
    except docker.errors.NotFound:
        print(f"Container '{container_name}' not found!")

# First, remove old containers if they exist
remove_container_if_exists("mysql")
remove_container_if_exists("adminer")

# Then, run new containers
run_mysql_container(network_name)
run_adminer_container(network_name)

# Show their IP addresses
print("\nGetting container IP addresses:")
show_container_ip("mysql", network_name)
show_container_ip("adminer", network_name)


Container 'mysql' already exists. Removing it...
Container 'adminer' already exists. Removing it...
Running MySQL container...
MySQL container started.
Running Adminer container...
Adminer container started.

Getting container IP addresses:
mysql IP Address: 172.18.0.2
adminer IP Address: 172.18.0.3
